In [143]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
import pickle

In [144]:
# loading data
data = pd.read_csv("Churn_Modelling.csv")
data = data[data["Exited"] != "Exited"]
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.8,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.1,0


In [145]:
# pre-process the data 
# remove the irrelevant data 
data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.8,3,1,0,113931.57,1
3,699,France,Female,39,1,0,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.1,0
...,...,...,...,...,...,...,...,...,...,...,...
19998,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
19999,709,France,Female,36,7,0,1,0,1,42085.58,1
20000,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
20001,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


In [146]:
# encode categorical vars
label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data['Gender'])
data


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.8,3,1,0,113931.57,1
3,699,France,0,39,1,0,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.1,0
...,...,...,...,...,...,...,...,...,...,...,...
19998,516,France,1,35,10,57369.61,1,1,1,101699.77,0
19999,709,France,0,36,7,0,1,0,1,42085.58,1
20000,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1
20001,792,France,0,28,4,130142.79,1,1,0,38190.78,0


In [ ]:
#  one hot encoding for column Geography
from sklearn.preprocessing import OneHotEncoder
onehotencoder_geography = OneHotEncoder()
geography_encoder = onehotencoder_geography.fit_transform(data[["Geography"]])
geography_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 20002 stored elements and shape (20002, 4)>

In [148]:
onehotencoder_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain',
       'Geography_nan'], dtype=object)

In [149]:
geo_encoded_df = pd.DataFrame(geography_encoder.toarray(),columns=onehotencoder_geography.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain,Geography_nan
0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0
...,...,...,...,...
19997,1.0,0.0,0.0,0.0
19998,1.0,0.0,0.0,0.0
19999,0.0,1.0,0.0,0.0
20000,1.0,0.0,0.0,0.0


In [150]:
data = pd.concat([data.drop('Geography',axis=1), geo_encoded_df],axis=1)

In [151]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Geography_nan
0,619,0.0,42,2,0,1,1,1,101348.88,1,1.0,0.0,0.0,0.0
1,608,0.0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0,0.0
2,502,0.0,42,8,159660.8,3,1,0,113931.57,1,1.0,0.0,0.0,0.0
3,699,0.0,39,1,0,2,0,0,93826.63,0,1.0,0.0,0.0,0.0
4,850,0.0,43,2,125510.82,1,1,1,79084.1,0,0.0,0.0,1.0,0.0


In [152]:
# save the encoder and scaler 
with open('label_encoder_gender.pkl' , 'wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehotencoder_geography.pkl' , 'wb') as file:
    pickle.dump(onehotencoder_geography , file)

In [156]:
# divide dataset into dependent and independent features 
X = data.drop('Exited',axis=1)
y = data['Exited']
y = pd.to_numeric(y, errors='coerce')
y = y.fillna(0).astype('float32')

for col in X.columns:
    X[col] = pd.to_numeric(X[col], errors='coerce')
X.fillna(0, inplace=True)

# Replace NaNs with 0 (or mean/median as appropriate)
X.fillna(0, inplace=True)

# split the data and train and test 
X_train , X_test  , y_train , y_test  = train_test_split( X , y ,test_size=0.2 , random_state=42)

# scaled out  
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# y_train and y_test must be NumPy arrays
y_train = y_train.values
y_test = y_test.values


print(X.dtypes)   # Make sure all are float64 or int64
print(X.head())   # Ensure no junk text


CreditScore          float64
Gender               float64
Age                  float64
Tenure               float64
Balance              float64
NumOfProducts        float64
HasCrCard            float64
IsActiveMember       float64
EstimatedSalary      float64
Geography_France     float64
Geography_Germany    float64
Geography_Spain      float64
Geography_nan        float64
dtype: object
   CreditScore  Gender   Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0        619.0     0.0  42.0     2.0       0.00            1.0        1.0   
1        608.0     0.0  41.0     1.0   83807.86            1.0        0.0   
2        502.0     0.0  42.0     8.0  159660.80            3.0        1.0   
3        699.0     0.0  39.0     1.0       0.00            2.0        0.0   
4        850.0     0.0  43.0     2.0  125510.82            1.0        1.0   

   IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
0             1.0        101348.88               1.0                0.0

In [157]:
print(X_train.dtype, X_train.shape)
print(type(y_train), y_train.dtype, y_train.shape)

float64 (16002, 13)
<class 'numpy.ndarray'> float32 (16002,)


In [158]:
X_train

array([[ 0.77949333,  0.91049939, -0.85107117, ...,  1.7307529 ,
        -0.57181789, -0.01118034],
       [-0.24282006,  0.91049939,  1.24674639, ...,  1.7307529 ,
        -0.57181789, -0.01118034],
       [-0.66620238,  0.91049939,  1.81887845, ..., -0.57778323,
        -0.57181789, -0.01118034],
       ...,
       [ 0.87243091, -1.09774475, -0.08822842, ..., -0.57778323,
        -0.57181789, -0.01118034],
       [ 0.17023585,  0.91049939,  0.3885483 , ..., -0.57778323,
        -0.57181789, -0.01118034],
       [ 0.19088865,  0.91049939,  1.72352311, ..., -0.57778323,
        -0.57181789, -0.01118034]])

In [159]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [160]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Geography_nan
0,619,0.0,42,2,0,1,1,1,101348.88,1,1.0,0.0,0.0,0.0
1,608,0.0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0,0.0
2,502,0.0,42,8,159660.8,3,1,0,113931.57,1,1.0,0.0,0.0,0.0
3,699,0.0,39,1,0,2,0,0,93826.63,0,1.0,0.0,0.0,0.0
4,850,0.0,43,2,125510.82,1,1,1,79084.1,0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19999,709,0.0,36,7,0,1,0,1,42085.58,1,0.0,1.0,0.0,0.0
20000,772,1.0,42,3,75075.31,2,1,0,92888.52,1,1.0,0.0,0.0,0.0
20001,792,0.0,28,4,130142.79,1,1,0,38190.78,0,0.0,0.0,0.0,1.0
20002,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ANN implementation

In [161]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime

In [162]:
# building the ann model 
model = Sequential([
    Dense(64, activation='relu', input_shape = (X_train.shape[1],)), ## 1st hidden layer 
    Dense(32, activation='relu'), ## 2nd hidden layer 
    Dense(1, activation='sigmoid') ## output layer 
])

In [163]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 64)                896       
                                                                 
 dense_22 (Dense)            (None, 32)                2080      
                                                                 
 dense_23 (Dense)            (None, 1)                 33        
                                                                 
Total params: 3,009
Trainable params: 3,009
Non-trainable params: 0
_________________________________________________________________


In [164]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

In [165]:
# compile the model
model.compile(optimizer = opt , loss=loss , metrics=['accuracy'])

In [166]:
# setup the TensorBoard
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir , histogram_freq=1)

In [167]:
# setup early stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss' ,patience= 10 , restore_best_weights=True)

In [168]:
data = data[data["Exited"] != "Exited"]  # Remove string rows
y = data["Exited"].astype('float32')
X = data.drop("Exited", axis=1)

In [169]:
y = data["Exited"].astype("float32")  # ensure it's float
X = data.drop("Exited", axis=1)

In [170]:
# Train the model 
history  = model.fit(
    X_train , y_train , validation_data = (X_test, y_test) , epochs = 10,
    callbacks = [tensorflow_callback]
)

Epoch 1/10
501/501 [==============================] - 1s 1ms/step - loss: 0.3947 - accuracy: 0.8275 - val_loss: 0.3539 - val_accuracy: 0.8618
Epoch 2/10
501/501 [==============================] - 1s 1ms/step - loss: 0.3635 - accuracy: 0.8528 - val_loss: 0.3538 - val_accuracy: 0.8610
Epoch 3/10
501/501 [==============================] - 1s 1ms/step - loss: 0.3516 - accuracy: 0.8560 - val_loss: 0.3519 - val_accuracy: 0.8623
Epoch 4/10
501/501 [==============================] - 1s 1ms/step - loss: 0.3477 - accuracy: 0.8561 - val_loss: 0.3483 - val_accuracy: 0.8615
Epoch 5/10
501/501 [==============================] - 1s 1ms/step - loss: 0.3454 - accuracy: 0.8578 - val_loss: 0.3531 - val_accuracy: 0.8638
Epoch 6/10
501/501 [==============================] - 1s 1ms/step - loss: 0.3432 - accuracy: 0.8575 - val_loss: 0.3433 - val_accuracy: 0.8620
Epoch 7/10
501/501 [==============================] - 1s 1ms/step - loss: 0.3419 - accuracy: 0.8596 - val_loss: 0.3471 - val_accuracy: 0.8635
Epoch 

In [172]:
model.save('model.keras')
print(model.loss)

In [175]:
# load TensorBoard
%reload_ext tensorboard

In [178]:
%tensorboard --logdir logs/fit 


Reusing TensorBoard on port 6006 (pid 23784), started 2 days, 22:14:59 ago. (Use '!kill 23784' to kill it.)